## 1. Load Splitted Data

In [5]:
import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)	
import matplotlib.pyplot as plt
import os
os.chdir(r'D:\ME\GeorgeTown\FALL2022\ANLY-590\ANLY590\Data')
import numpy as np
import scipy.sparse
from tqdm import tqdm
import functools

In [2]:
#Load in splitted data
X_train=pd.read_csv(r'data_splitted\X_train_splitted.csv')
y_train=pd.read_csv(r'data_splitted\y_train_splitted.csv')
X_test=pd.read_csv(r'data_splitted\X_test_splitted.csv')
y_test=pd.read_csv(r'data_splitted\y_test_splitted.csv')

In [3]:
X_train.head()

,posting_id,image,image_phash,title
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE"
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml


In [4]:
print('Training Size:',X_train.shape)
print('Test Size:',X_test.shape)

Training Size: (34085, 4)
Test Size: (165, 4)


## 2. TF-IDF KNN

In [17]:
#Load in tf-idf features
X_train_tfidf=scipy.sparse.load_npz(r"TFIDF\X_train_tfidf.npz")
X_test_tfidf=scipy.sparse.load_npz(r"TFIDF\X_test_tfidf.npz")

In [18]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_tfidf,y_train)

D:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [22]:
#Get top5 similar post other than itself
k=6
dis,idx=knn.kneighbors(X_train_tfidf[:,:],n_neighbors=k)

dis_idx=np.array(list(zip(dis.flatten(),idx.flatten()))).reshape(-1,k,2).tolist()
for i in tqdm(range(len(dis_idx))):
    sign=0
    for j in range(5):
        if int(dis_idx[i][j][1])==i:
            dis_idx[i].pop(j)
            sign=1
            break
    if not sign:dis_idx[i].pop(-1)

100%|████████████████████████████████████████████████████████████████████████| 34085/34085 [00:00<00:00, 362618.87it/s]


In [23]:
dis_idx=np.array(dis_idx).reshape(-1,2)
dis_idx=pd.DataFrame({'post_idx':functools.reduce(lambda x,y:x+y,[[i]*5 for i in range(X_train.shape[0])],[]),
            'top_similar':list(range(1,6))*X_train.shape[0],
            'similar_idx':dis_idx[:,1],
            'distance':dis_idx[:,0]})
dis_idx.similar_idx=dis_idx.similar_idx.astype(int)
dis_idx.head(20)

,post_idx,top_similar,similar_idx,distance
0,0,1,32996,0.000000
1,0,2,9056,1.179860
2,0,3,30994,1.214102
3,0,4,25221,1.217782
4,0,5,2565,1.234942
5,1,1,19097,1.137798
6,1,2,10281,1.137798
7,1,3,21625,1.137798
8,1,4,23158,1.158956
9,1,5,25034,1.169706


In [24]:
dis_idx.to_csv(r'KNN\text_similarity_train.csv',index=False)

## Evaluate Text Similarity Only Top5 Accuracy

In [15]:
dis_idx['neighbor_label']=dis_idx['similar_idx'].apply(lambda x:y_train.iloc[x])
dis_idx['label']=dis_idx['post_idx'].apply(lambda x:y_train.iloc[x])
dis_idx.head(10)

,post_idx,top_similar,similar_idx,distance,neighbor_label,label
0,0,1,32996,0.000000,249114794,249114794
1,0,2,9056,1.179860,471581622,249114794
2,0,3,30994,1.214102,471581622,249114794
3,0,4,25221,1.217782,866425052,249114794
4,0,5,2565,1.234942,982607408,249114794
5,1,1,19097,1.137798,475342649,2937985045
6,1,2,10281,1.137798,475342649,2937985045
7,1,3,21625,1.137798,475342649,2937985045
8,1,4,23158,1.158956,4148134568,2937985045
9,1,5,25034,1.169706,2819310070,2937985045


In [18]:
#Training Accuracy
found=0
for i in tqdm(range(X_train.shape[0])):
    for j in range(i*5,i*5+5):
        if dis_idx.neighbor_label.iloc[j]==dis_idx.label.iloc[j]:
            found+=1
            break
print('Top5 Train Accuracy with Only Text Similarity:',found/X_train.shape[0]*100,'%')

100%|██████████████████████████████████████████████████████████████████████████| 34085/34085 [00:04<00:00, 7983.43it/s]

Top5 Train Accuracy with Only Text Similarity: 83.14214463840399 %


In [21]:
#Get top5 similar post other than itself on test set
k=6
dis,idx=knn.kneighbors(X_test_tfidf[:,:],n_neighbors=k)

dis_idx=np.array(list(zip(dis.flatten(),idx.flatten()))).reshape(-1,k,2).tolist()
for i in tqdm(range(len(dis_idx))):
    sign=0
    for j in range(5):
        if int(dis_idx[i][j][1])==i:
            dis_idx[i].pop(j)
            sign=1
            break
    if not sign:dis_idx[i].pop(-1)

dis_idx=np.array(dis_idx).reshape(-1,2)
dis_idx=pd.DataFrame({'post_idx':functools.reduce(lambda x,y:x+y,[[i]*5 for i in range(X_test.shape[0])],[]),
            'top_similar':list(range(1,6))*X_test.shape[0],
            'similar_idx':dis_idx[:,1],
            'distance':dis_idx[:,0]})
dis_idx.similar_idx=dis_idx.similar_idx.astype(int)

dis_idx.to_csv(r'KNN\text_similarity_test.csv',index=False)

dis_idx['neighbor_label']=dis_idx['similar_idx'].apply(lambda x:y_train.iloc[x])
dis_idx['label']=dis_idx['post_idx'].apply(lambda x:y_test.iloc[x])

#Test Accuracy
found=0
for i in tqdm(range(X_test.shape[0])):
    for j in range(i*5,i*5+5):
        if dis_idx.neighbor_label.iloc[j]==dis_idx.label.iloc[j]:
            found+=1
            break
print('Top5 Test Accuracy with Only Text Similarity:',found/X_test.shape[0]*100,'%')

100%|█████████████████████████████████████████████████████████████████████████████| 165/165 [00:00<00:00, 27332.55it/s]

Top5 Test Accuracy with Only Text Similarity: 97.57575757575758 %


## 3.CV-KNN

In [6]:
#Load in cv vectors
X_train_cv=np.load(r"CVvector\X_train_prob_matrix.npy")
X_test_cv=np.load(r"CVvector\X_test_prob_matrix.npy")

In [7]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_cv,y_train)

D:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [8]:
#Get top5 similar post other than itself
k=6
dis,idx=knn.kneighbors(X_train_cv[:,:],n_neighbors=k)

dis_idx=np.array(list(zip(dis.flatten(),idx.flatten()))).reshape(-1,k,2).tolist()
for i in tqdm(range(len(dis_idx))):
    sign=0
    for j in range(5):
        if int(dis_idx[i][j][1])==i:
            dis_idx[i].pop(j)
            sign=1
            break
    if not sign:dis_idx[i].pop(-1)

100%|████████████████████████████████████████████████████████████████████████| 34085/34085 [00:00<00:00, 558778.23it/s]


In [9]:
dis_idx=np.array(dis_idx).reshape(-1,2)
dis_idx=pd.DataFrame({'post_idx':functools.reduce(lambda x,y:x+y,[[i]*5 for i in range(X_train.shape[0])],[]),
            'top_similar':list(range(1,6))*X_train.shape[0],
            'similar_idx':dis_idx[:,1],
            'distance':dis_idx[:,0]})
dis_idx.similar_idx=dis_idx.similar_idx.astype(int)
dis_idx.head(20)

,post_idx,top_similar,similar_idx,distance
0,0,1,32996,0.743013
1,0,2,27535,0.964640
2,0,3,30258,0.969116
3,0,4,23990,0.969938
4,0,5,3061,0.971571
5,1,1,445,0.000042
6,1,2,180,0.000062
7,1,3,25034,0.000063
8,1,4,18518,0.000072
9,1,5,23428,0.000169


In [10]:
dis_idx.to_csv(r'KNN\img_similarity_train.csv',index=False)

## Evaluate Text Similarity Only Top5 Accuracy

In [11]:
dis_idx['neighbor_label']=dis_idx['similar_idx'].apply(lambda x:y_train.iloc[x])
dis_idx['label']=dis_idx['post_idx'].apply(lambda x:y_train.iloc[x])
dis_idx.head(10)

,post_idx,top_similar,similar_idx,distance,neighbor_label,label
0,0,1,32996,0.743013,249114794,249114794
1,0,2,27535,0.964640,656698835,249114794
2,0,3,30258,0.969116,1071020646,249114794
3,0,4,23990,0.969938,2925334956,249114794
4,0,5,3061,0.971571,2008989859,249114794
5,1,1,445,0.000042,2819310070,2937985045
6,1,2,180,0.000062,2819310070,2937985045
7,1,3,25034,0.000063,2819310070,2937985045
8,1,4,18518,0.000072,2937985045,2937985045
9,1,5,23428,0.000169,2819310070,2937985045


In [12]:
#Training Accuracy
found=0
for i in tqdm(range(X_train.shape[0])):
    for j in range(i*5,i*5+5):
        if dis_idx.neighbor_label.iloc[j]==dis_idx.label.iloc[j]:
            found+=1
            break
print('Top5 Train Accuracy with Only Image Similarity:',found/X_train.shape[0]*100,'%')

100%|█████████████████████████████████████████████████████████████████████████| 34085/34085 [00:01<00:00, 18124.16it/s]

Top5 Train Accuracy with Only Image Similarity: 86.92093296171336 %


In [16]:
#Get top5 similar post other than itself on test set
k=6
dis,idx=knn.kneighbors(X_test_cv[:,:],n_neighbors=k)

dis_idx=np.array(list(zip(dis.flatten(),idx.flatten()))).reshape(-1,k,2).tolist()
for i in tqdm(range(len(dis_idx))):
    sign=0
    for j in range(5):
        if int(dis_idx[i][j][1])==i:
            dis_idx[i].pop(j)
            sign=1
            break
    if not sign:dis_idx[i].pop(-1)

dis_idx=np.array(dis_idx).reshape(-1,2)
dis_idx=pd.DataFrame({'post_idx':functools.reduce(lambda x,y:x+y,[[i]*5 for i in range(X_test.shape[0])],[]),
            'top_similar':list(range(1,6))*X_test.shape[0],
            'similar_idx':dis_idx[:,1],
            'distance':dis_idx[:,0]})
dis_idx.similar_idx=dis_idx.similar_idx.astype(int)

dis_idx.to_csv(r'KNN\img_similarity_test.csv',index=False)

dis_idx['neighbor_label']=dis_idx['similar_idx'].apply(lambda x:y_train.iloc[x])
dis_idx['label']=dis_idx['post_idx'].apply(lambda x:y_test.iloc[x])

#Test Accuracy
found=0
for i in tqdm(range(X_test.shape[0])):
    for j in range(i*5,i*5+5):
        if dis_idx.neighbor_label.iloc[j]==dis_idx.label.iloc[j]:
            found+=1
            break
print('Top5 Test Accuracy with Only Image Similarity:',found/X_test.shape[0]*100,'%')

100%|█████████████████████████████████████████████████████████████████████████████| 165/165 [00:00<00:00, 13755.37it/s]

Top5 Test Accuracy with Only Image Similarity: 65.45454545454545 %
